In [ ]:
import duckdb
import pandas as pd

##TODO: Falta bajar el dataset directamente de Kaggle
# import kagglehub
# from kagglehub import KaggleDatasetAdapter

# # Load the dataset from KaggleHub
# df = kagglehub.load_dataset(
#     KaggleDatasetAdapter.PANDAS,
#     "alejandroczernikier/properati-argentina-dataset",
#     file_path="/data/raw"
# )




con = duckdb.connect("data/DB/entrenamiento.duckdb")

# guardar CSV como Parquet
con.execute("""
COPY (
    SELECT * FROM read_csv_auto('data/RAW/entrenamiento.csv')
) TO 'data/parquet/entrenamiento.parquet' (FORMAT PARQUET)
""")


# Crear tabla desde el archivo Parquet
# con.execute("""
#     CREATE TABLE entrenamiento AS
#     SELECT * FROM 'data/parquet/entrenamiento.parquet'
# """)



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# cant. de filas
n_filas = duckdb.query("SELECT COUNT(*) FROM 'data/parquet/entrenamiento.parquet'").fetchone()[0]
print("🔢 Filas:", n_filas)


🔢 Filas: 992192


In [37]:

#profiling del dataset en parquet
parquet_file = "data/parquet/entrenamiento.parquet"

con = duckdb.connect()

# Obtener metadatos de columnas
cols = con.execute(f"""
DESCRIBE SELECT * FROM parquet_scan('{parquet_file}')
""").fetchdf()

profiling_results = []

for _, row in cols.iterrows():
    col = row["column_name"]
    dtype = row["column_type"].lower()

    is_numeric = any(t in dtype for t in ["int", "decimal", "double", "float", "real"])

    stats_query = f"""
    SELECT
        '{col}' AS column,
        '{dtype}' AS dtype,
        COUNT(*) AS total,
        COUNT(*) FILTER (WHERE "{col}" IS NULL) AS nulls,
        COUNT(DISTINCT "{col}") AS distinct_values,
        { "MIN(" + col + ")" if is_numeric else "NULL" } AS min,
        { "MAX(" + col + ")" if is_numeric else "NULL" } AS max,
        { "AVG(" + col + ")" if is_numeric else "NULL" } AS mean,
        { "MEDIAN(" + col + ")" if is_numeric else "NULL" } AS median,
        { "QUANTILE(" + col + ", 0.25)" if is_numeric else "NULL" } AS Q1,
        { "QUANTILE(" + col + ", 0.75)" if is_numeric else "NULL" } AS Q3
    FROM parquet_scan('{parquet_file}')
    """

    stats = con.execute(stats_query).fetchdf()
    profiling_results.append(stats)

profile_df = pd.concat(profiling_results)

print("profiling completado")
display(profile_df)


profiling completado


C:\Users\beln3\AppData\Local\Temp\ipykernel_18400\3411417528.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  profile_df = pd.concat(profiling_results)


,column,dtype,total,nulls,distinct_values,min,max,mean,median,Q1,Q3
0,id,bigint,992192,0,992192,1.000000,1.000000e+06,500338.438758,500641.500000,250259.000000,750507.000000
0,ad_type,varchar,992192,0,1,NaN,NaN,NaN,NaN,NaN,NaN
0,start_date,date,992192,0,346,NaN,NaN,NaN,NaN,NaN,NaN
0,end_date,date,992192,0,450,NaN,NaN,NaN,NaN,NaN,NaN
0,created_on,date,992192,0,346,NaN,NaN,NaN,NaN,NaN,NaN
0,lat,double,992192,153198,317046,-180.000000,-3.504503e+01,-59.266586,-58.444383,-58.756011,-58.058966
0,lon,double,992192,152263,315745,-54.842464,8.505113e+01,-34.793745,-34.610508,-34.921709,-34.459156
0,l1,varchar,992192,0,4,NaN,NaN,NaN,NaN,NaN,NaN
0,l2,varchar,992192,0,42,NaN,NaN,NaN,NaN,NaN,NaN
0,l3,varchar,992192,57455,1262,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df = duckdb.query("""
    SELECT *
    FROM 'data/parquet/entrenamiento.parquet'
    WHERE l2 = 'Capital Federal'
""").df()

In [58]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249738 entries, 0 to 249737
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               249738 non-null  int64         
 1   ad_type          249738 non-null  object        
 2   start_date       249738 non-null  datetime64[us]
 3   end_date         249738 non-null  datetime64[us]
 4   created_on       249738 non-null  datetime64[us]
 5   lat              217583 non-null  float64       
 6   lon              217705 non-null  float64       
 7   l1               249738 non-null  object        
 8   l2               249738 non-null  object        
 9   l3               225456 non-null  object        
 10  l4               10091 non-null   object        
 11  l5               0 non-null       object        
 12  l6               0 non-null       object        
 13  rooms            171449 non-null  float64       
 14  bedrooms         122

In [ ]:
#profile del df solo de capital
def profile(df: pd.DataFrame):
    profile = pd.DataFrame({
        "dtype": df.dtypes,
        "n_nulls": df.isnull().sum(),
        "n_unique": df.nunique(),
        "min": df.min(numeric_only=True),
        "Q1": df.quantile(0.25, numeric_only=True),
        "median": df.median(numeric_only=True),
        "mean": df.mean(numeric_only=True),
        "Q3": df.quantile(0.75, numeric_only=True),
        "max": df.max(numeric_only=True)
    })
    return profile

profile_df = profile(df)
display(profile_df)


,dtype,n_nulls,n_unique,min,Q1,median,mean,Q3,max
ad_type,object,0,1,NaN,NaN,NaN,NaN,NaN,NaN
bathrooms,float64,37245,18,1.000000,1.000000,1.000000,1.545119,2.000000,2.000000e+01
bedrooms,float64,127386,65,-2.000000,1.000000,2.000000,1.951525,3.000000,3.030000e+02
created_on,datetime64[us],0,346,NaN,NaN,NaN,NaN,NaN,NaN
currency,object,8291,2,NaN,NaN,NaN,NaN,NaN,NaN
description,object,0,156677,NaN,NaN,NaN,NaN,NaN,NaN
end_date,datetime64[us],0,448,NaN,NaN,NaN,NaN,NaN,NaN
id,int64,0,249738,5.000000,216169.000000,463253.500000,469234.024125,693799.000000,1.000000e+06
l1,object,0,1,NaN,NaN,NaN,NaN,NaN,NaN
l2,object,0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#contenido de columnas
for col in ["l1", "l2", "l3", "l4", "l5", "l6"]:
    print(f"\n Valores únicos en {col}:")
    print(df[col].dropna().unique())



 Valores únicos en l1:
['Argentina']

 Valores únicos en l2:
['Capital Federal']

 Valores únicos en l3:
['Colegiales' 'Almagro' 'Villa Urquiza' 'Boedo' 'San Telmo' 'Barrio Norte'
 'Villa Devoto' 'Palermo' 'Monserrat' 'Caballito' 'Balvanera'
 'Villa Ortuzar' 'Villa Crespo' 'Once' 'Saavedra' 'Recoleta'
 'Villa General Mitre' 'Villa del Parque' 'Belgrano' 'Mataderos'
 'Floresta' 'Coghlan' 'Flores' 'Velez Sarsfield' 'Parque Chacabuco'
 'Nuñez' 'Constitución' 'Las Cañitas' 'Villa Riachuelo' 'Puerto Madero'
 'Paternal' 'Congreso' 'Parque Centenario' 'Parque Patricios' 'Pompeya'
 'Villa Lugano' 'San Cristobal' 'Villa Real' 'San Nicolás' 'Boca'
 'Versalles' 'Villa Luro' 'Retiro' 'Chacarita' 'Barracas' 'Monte Castro'
 'Villa Pueyrredón' 'Parque Avellaneda' 'Liniers' 'Abasto' 'Tribunales'
 'Centro / Microcentro' 'Villa Santa Rita' 'Agronomía' 'Parque Chas'
 'Villa Soldati' 'Catalinas' 'Distrito Audiovisual']

 Valores únicos en l4:
['Palermo Viejo' 'Palermo Hollywood' 'Palermo Soho' 'Palermo C

In [61]:
#sacar columnas innecesarias
df = df.drop(columns=["title", "description", "ad_type", "id", "l1", "l2", "l4", "l5", "l6"])
df.head()

,start_date,end_date,created_on,lat,lon,l3,rooms,bedrooms,bathrooms,surface_total,surface_covered,currency,price_period,property_type,operation_type,price
0,2019-11-29,9999-12-31,2019-11-29,-58.442399,-34.573623,Colegiales,3.0,2.0,2.0,NaN,NaN,USD,None,Departamento,Venta,259000.0
1,2020-06-05,2020-06-08,2020-06-05,-58.430493,-34.606620,Almagro,3.0,2.0,2.0,77.0,67.0,USD,None,Departamento,Venta,235500.0
2,2020-07-01,9999-12-31,2020-07-01,-58.491760,-34.574123,Villa Urquiza,2.0,NaN,1.0,60.0,55.0,USD,None,Departamento,Venta,175000.0
3,2019-08-17,9999-12-31,2019-08-17,-58.420737,-34.631770,Boedo,2.0,1.0,1.0,74.0,47.0,USD,None,PH,Venta,140000.0
4,2019-08-05,2019-08-31,2019-08-05,-58.429983,-34.607225,Almagro,3.0,2.0,1.0,66.0,64.0,USD,None,Departamento,Venta,173000.0


In [62]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249738 entries, 0 to 249737
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   start_date       249738 non-null  datetime64[us]
 1   end_date         249738 non-null  datetime64[us]
 2   created_on       249738 non-null  datetime64[us]
 3   lat              217583 non-null  float64       
 4   lon              217705 non-null  float64       
 5   l3               225456 non-null  object        
 6   rooms            171449 non-null  float64       
 7   bedrooms         122352 non-null  float64       
 8   bathrooms        212493 non-null  float64       
 9   surface_total    171780 non-null  float64       
 10  surface_covered  165345 non-null  float64       
 11  currency         241447 non-null  object        
 12  price_period     99785 non-null   object        
 13  property_type    249738 non-null  object        
 14  operation_type   249